In [1]:
import mysql_connection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import joblib


def fetch_data(team, players):
    connection = mysql_connection.get_mysql_connection()

    if connection:
        dfs = []

        for player in players:
            if team == 'All' and player == 'All':
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.overs, b.runs, b.wickets, b.economy_rate, b.dot, m.ground, m.match_date
                FROM cricket_info.bowling AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id;
                """
                cursor = connection.cursor()
                cursor.execute(sql_query)
            elif team == 'All':
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.overs, b.runs, b.wickets, b.economy_rate, b.dot, m.ground, m.match_date
                FROM cricket_info.bowling AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id
                WHERE b.player LIKE %s;
                """
                cursor = connection.cursor()
                cursor.execute(sql_query, (f'%{player}%',))
            else:
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.overs, b.runs, b.wickets, b.economy_rate, b.dot, m.ground, m.match_date
                FROM cricket_info.bowling AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id
                WHERE b.team = %s AND b.player LIKE %s;
                """
                cursor = connection.cursor()
                cursor.execute(sql_query, (team, f'%{player}%'))

           
            rows = cursor.fetchall()
            df = pd.DataFrame(rows, columns=['match_id', 'team', 'opposite_team', 'player', 'overs', 'runs', 'wickets', 'economy_rate', 'dot', 'ground', 'date'])
            dfs.append(df)
            cursor.close()

        connection.close()
        df = pd.concat(dfs, ignore_index=True)
        return df
    else:
        print("Connection to the database failed.")
        return None


# Function to convert string representation of overs to decimal
def convert_to_decimal(overs_str):
    if '.' in overs_str:
        overs, balls = overs_str.split('.')
        decimal_overs = float(overs) + float(balls) / 6
    else:
        decimal_overs = float(overs_str)
    return decimal_overs


# Function to calculate Bowling Average
def calculate_bowling_average(runs, wickets):
    if wickets == 0:
        return 0
    else:
        return runs / wickets


# Function to calculate Economy Rate
def calculate_economy_rate(runs, overs):
    if overs == 0:
        return 0
    else:
        return runs / overs


# Function to calculate Strike Rate
def calculate_strike_rate(overs, wickets):
    if wickets == 0:
        return 0
    else:
        return (overs * 6) / wickets


# Function to calculate Wicket-taking Ability (Wickets per Match)
def calculate_wickets_per_match(wickets, matches):
    if wickets == 0:
        return 0
    else:
        return wickets / matches


# Function to calculate Wicket-taking Ability (Wickets per Over)
def calculate_wickets_per_over(wickets, overs):
    if wickets == 0:
        return 0
    else:
        return wickets / overs
    
def calculate_dots_per_over(dot, overs):
    if dot == 0:
        return 0
    else:
        return dot / overs
    

def transform_data(df):
    
    # Convert numeric columns to appropriate data types
    df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y', errors='coerce')

    # df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')
    df['runs'] = df['runs'].astype(float)
    df['overs'] = df['overs'].apply(convert_to_decimal)
    df['wickets'] = df['wickets'].astype(float)
    
    # Replace '-' with NaN
    df['dot'].replace('-', np.nan, inplace=True)
    # Convert to float, skipping NaN values
    df['dot'] = pd.to_numeric(df['dot'], errors='coerce')
    # Fill NaN values with 0
    df['dot'].fillna(0, inplace=True)
    # Convert to float explicitly
    df['dot'] = df['dot'].astype(float)

    # Calculate Bowling Average
    df['bowling_average'] = df.apply(lambda row: calculate_bowling_average(row['runs'], row['wickets']), axis=1)
    
    # Calculate Economy Rate
    df['economy_rate'] = df.apply(lambda row: calculate_economy_rate(row['runs'], row['overs']), axis=1)
    
    # Calculate Strike Rate
    df['strike_rate'] = df.apply(lambda row: calculate_strike_rate(row['overs'], row['wickets']), axis=1)
        
    # Calculate Wickets per Over
    df['wickets_per_over'] = df.apply(lambda row: calculate_wickets_per_over(row['wickets'], row['overs']), axis=1)

    # Calculate Wickets per Over
    df['dots_per_over'] = df.apply(lambda row: calculate_dots_per_over(row['dot'], row['overs']), axis=1)


    # Assuming 'date' column is in datetime format
    try:
        df['day'] = df['date'].dt.day
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
    except ValueError:
        # If ValueError occurs, it means there are records with incorrect date format
        print("Dropping records with incorrect date format...")
        # Get indices of records with incorrect date format
        wrong_date_indices = master_df.index[(df['date'].dt.day.isna()) | (df['date'].dt.month.isna()) | (df['date'].dt.year.isna())]
        # Drop records with incorrect date format
        df.drop(wrong_date_indices, inplace=True)
        print(f"Dropped {len(wrong_date_indices)} records.")

    
    # Define and populate match_stats if needed
    # match_stats = ...

    return df



def models_evaluate(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Drop records with non-numeric values in 'day', 'month', or 'year' columns
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')
    
    # Convert 'day', 'month', and 'year' columns to numeric
    master_df['day'] = pd.to_numeric(master_df['day'], errors='coerce')
    master_df['month'] = pd.to_numeric(master_df['month'], errors='coerce')
    master_df['year'] = pd.to_numeric(master_df['year'], errors='coerce')
    
    # Drop records with NaN values after conversion
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')

    # Ensure 'team', 'opposite_team', and 'ground' columns are of string data type
    master_df[['team', 'opposite_team', 'ground']] = master_df[['team', 'opposite_team', 'ground']].astype(str)
    
    # Remove records with non-numeric or NaN values in input features or target variable
    numeric_features = ['overs', 'runs', 'wickets', 'economy_rate', 'dot', 'day', 'month', 'year', 'bowling_average', 'strike_rate', 'wickets_per_over', 'dots_per_over']
    master_df = master_df.dropna(subset=numeric_features + ['wickets'], how='any')

    # Splitting the data into features (X) and target variable (y)
    X = master_df[['team', 'opposite_team', 'overs', 'runs', 'wickets', 'economy_rate', 'dot', 'ground', 'day', 'month', 'year', 'bowling_average', 'strike_rate', 'wickets_per_over', 'dots_per_over']]
    y = master_df['wickets']
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric and categorical transformers
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, ['team', 'opposite_team', 'ground'])])
    
    # Initialize regression models
    models = [
        ('Linear Regression', LinearRegression()),
        ('Support Vector Regression', SVR()),
        ('Random Forest Regression', RandomForestRegressor(random_state=42)),
        ('Gradient Boosting Regression', GradientBoostingRegressor(random_state=42)),
        ('Decision Tree Regression', DecisionTreeRegressor(random_state=42)),
    ]

    # Train and evaluate models
    for name, model in models:
        regressor = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
        regressor.fit(X_train, y_train)
        predictions = regressor.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        rmse = mse ** 0.5
        r2 = r2_score(y_test, predictions)
        print(f'{name}:')
        print(f'Mean Squared Error: {mse}')
        print(f'Root Mean Squared Error: {rmse}')
        print(f'R-squared: {r2}')
        print('\n')


def model_implement(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Drop records with non-numeric values in 'day', 'month', or 'year' columns
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')
    
    # Convert 'day', 'month', and 'year' columns to numeric
    master_df['day'] = pd.to_numeric(master_df['day'], errors='coerce')
    master_df['month'] = pd.to_numeric(master_df['month'], errors='coerce')
    master_df['year'] = pd.to_numeric(master_df['year'], errors='coerce')
    
    # Drop records with NaN values after conversion
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')

    # Ensure 'team', 'opposite_team', and 'ground' columns are of string data type
    master_df[['team', 'opposite_team', 'ground']] = master_df[['team', 'opposite_team', 'ground']].astype(str)
    
    # Remove records with non-numeric or NaN values in input features or target variable
    numeric_features = ['overs', 'runs', 'wickets', 'economy_rate', 'dot', 'day', 'month', 'year', 'bowling_average', 'strike_rate', 'wickets_per_over', 'dots_per_over']
    master_df = master_df.dropna(subset=numeric_features + ['wickets'], how='any')

    # Splitting the data into features (X) and target variable (y)
    X = master_df[['team', 'opposite_team', 'overs', 'runs', 'wickets', 'economy_rate', 'dot', 'ground', 'day', 'month', 'year', 'bowling_average', 'strike_rate', 'wickets_per_over', 'dots_per_over']]
    y = master_df['wickets']
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric and categorical transformers
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, ['team', 'opposite_team', 'ground'])])
    
    # Initialize and train Gradient Boosting Regression model
    model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))])
    model.fit(X_train, y_train)
    
    # Save the trained model for future prediction
    joblib.dump(model, 'gradient_boosting_model_for_bowling.pkl')

    print('Save the trained model for future prediction')





if __name__ == "__main__":
    team = 'All'
    players = ['All']
    # team = 'New Zealand'
    # players = ['Tim Southee', 'Ish Sodhi']
    
    df = fetch_data(team, players)
    print(df)
    transformed_df = transform_data(df)
    print(transformed_df)
    print(transformed_df.columns)
    models_evaluate(transformed_df)
    model_implement(transformed_df)

2024-05-04 17:18:06.949912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Connected to MySQL database
          match_id         team opposite_team               player overs runs  \
0         T20I # 3  New Zealand  South Africa           Shane Bond   3.3   25   
1         T20I # 3  New Zealand  South Africa           Kyle Mills     4   38   
2         T20I # 3  New Zealand  South Africa          Andre Adams     4   27   
3         T20I # 3  New Zealand  South Africa         Nathan Astle     4   20   
4         T20I # 3  New Zealand  South Africa         Jeetan Patel     4   20   
...            ...          ...           ...                  ...   ...  ...   
30052  T20I # 2427  Afghanistan        U.A.E.     Aayan Afzal Khan     4   26   
30053  T20I # 2427  Afghanistan        U.A.E.  Muhammad Jawadullah     4   30   
30054  T20I # 2427  Afghanistan        U.A.E.      Junaid Siddique   3.3   32   
30055  T20I # 2427  Afghanistan        U.A.E.           Ali Naseer     4   24   
30056  T20I # 2427  Afghanistan        U.A.E.       Dhruv Parashar     3   14   
